# Convolutional Neural Network

### Importing the libraries

In [177]:
!pip install --upgrade tensorflow keras Pillow

Requirement already up-to-date: tensorflow in /home/diego/anaconda3/envs/ml/lib/python3.7/site-packages (2.3.1)
Requirement already up-to-date: keras in /home/diego/anaconda3/envs/ml/lib/python3.7/site-packages (2.4.3)
Requirement already up-to-date: Pillow in /home/diego/anaconda3/envs/ml/lib/python3.7/site-packages (7.2.0)


In [178]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [179]:
tf.__version__

'2.3.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [180]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [181]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [182]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [183]:
cnn.add(tf.keras.layers.Conv2D(filters=50, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [184]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [185]:
cnn.add(tf.keras.layers.Conv2D(filters=50, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a third convolutional layer

In [186]:
cnn.add(tf.keras.layers.Conv2D(filters=50, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [187]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [188]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [189]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [190]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [148]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 77s 307ms/step - loss: 0.6721 - accuracy: 0.5735 - val_loss: 0.6519 - val_accuracy: 0.6025
Epoch 2/25
250/250 [==============================] - 75s 302ms/step - loss: 0.6142 - accuracy: 0.6664 - val_loss: 0.6017 - val_accuracy: 0.6685
Epoch 3/25
250/250 [==============================] - 75s 298ms/step - loss: 0.5633 - accuracy: 0.7101 - val_loss: 0.5447 - val_accuracy: 0.7190
Epoch 4/25
250/250 [==============================] - 76s 303ms/step - loss: 0.5424 - accuracy: 0.7249 - val_loss: 0.5453 - val_accuracy: 0.7225
Epoch 5/25
250/250 [==============================] - 75s 301ms/step - loss: 0.4955 - accuracy: 0.7525 - val_loss: 0.5121 - val_accuracy: 0.7475
Epoch 6/25
250/250 [==============================] - 75s 300ms/step - loss: 0.4904 - accuracy: 0.7667 - val_loss: 0.4762 - val_accuracy: 0.7865
Epoch 7/25
250/250 [==============================] - 76s 306ms/step - loss: 0.4679 - accuracy: 0.7764 - val_loss: 0.4710 - val_ac

## Part 4 - Making a single prediction

In [191]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [192]:
print(prediction)

dog


## Fim